In [1]:
import math
import random
from collections import deque, namedtuple

import gym
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from binance import Client
from gym import spaces

# Инициализация клиента Binance (публичные данные без API-ключей)
client = Client()


# Загрузка исторических данных с Binance
def get_klines(
    symbol="ETHUSDT",
    interval="1m",
    start_date="6 day ago UTC",
    end_date="2 day ago UTC",
):
    try:
        klines = client.get_historical_klines(
            symbol=symbol, interval=interval, start_str=start_date, end_str=end_date
        )
        # Преобразование результатов в DataFrame
        columns = [
            "time",
            "open",
            "high",
            "low",
            "close",
            "volume",
            "close_time",
            "quote_volume",
            "trades",
            "taker_buy_base",
            "taker_buy_quote",
            "ignore",
        ]
        df = pd.DataFrame(klines, columns=columns, dtype=float)
        # Конвертация времени в читаемый формат
        df["time"] = pd.to_datetime(df["time"], unit="ms")
        return df[["time", "open", "high", "low", "close", "volume", "trades"]]
    except Exception as e:
        print(f"Ошибка при получении данных: {e}")
        return None


# Получаем исторические данные (пример: последние 6 дней с интервалом 1 минута, до 2 дней назад)
df = get_klines(symbol="ETHUSDT", interval="1m")
print(df.head())  # Вывод первых строк для проверки загрузки данных

# Разделение данных на обучающую и тестовую выборки
if df is not None:
    N = len(df)
    ratio = 0.8
    df_test = df[int(N * ratio) :].reset_index(drop=True)
    df = df[: int(N * ratio)].reset_index(drop=True)
else:
    # В случае отсутствия данных, инициализируем пустые DataFrame
    df = pd.DataFrame(
        columns=["time", "open", "high", "low", "close", "volume", "trades"]
    )
    df_test = pd.DataFrame(
        columns=["time", "open", "high", "low", "close", "volume", "trades"]
    )


                 time     open     high      low    close     volume  trades
0 2025-03-17 15:21:00  1909.42  1910.89  1909.05  1910.41   120.9954  1212.0
1 2025-03-17 15:22:00  1910.40  1912.58  1909.60  1911.46   245.0425  1565.0
2 2025-03-17 15:23:00  1911.46  1911.77  1910.04  1911.23    99.0537  1030.0
3 2025-03-17 15:24:00  1911.22  1913.89  1911.06  1912.63  1619.1776  2335.0
4 2025-03-17 15:25:00  1912.63  1914.43  1912.20  1913.68   888.7267  2876.0


### Среда

In [2]:
class MarketMakingEnv(gym.Env):
    """
    Окружение маркет-мейкинга по Avellaneda-Stoikov (без учета задержки исполнения).
    Состояние: [inventory, относительное изменение цены, PnL].
    Действие: выбор пары индексов (γ, κ) из заданных списков gamma_values и kappa_values.
    """

    def __init__(self, df, gamma_values, kappa_values, T=60.0):
        super(MarketMakingEnv, self).__init__()
        self.prices = df.close.values
        self.prices_high = df.high.values
        self.prices_low = df.low.values
        self.N = len(df)
        self.gamma_values = gamma_values  # дискретный набор допустимых γ
        self.kappa_values = kappa_values  # дискретный набор допустимых κ
        self.T = T  # торговый горизонт (в секундах) для расчёта оптимальных спредов
        # Пространство действий: MultiDiscrete для пары индексов [index_gamma, index_kappa]
        self.action_space = spaces.MultiDiscrete([len(gamma_values), len(kappa_values)])
        # Пространство состояний: содержит инвентарь, относительное изменение цены и накопленный PnL
        high = np.array([100.0, 1.0, 1e9], dtype=np.float32)
        low = np.array([-100.0, -1.0, -1e9], dtype=np.float32)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)
        # Прочие параметры окружения
        self.initial_cash = 10000.0
        self.fee_rate = 0.0  # комиссия (можно изменить при необходимости)

    def reset(self):
        # Сброс состояния среды в начало эпизода
        self.t = 0
        self.inventory = 0.0
        self.cash = self.initial_cash
        # Начальное наблюдение: [инвентарь, 0 изменение цены, 0 PnL]
        return np.array([self.inventory, 0.0, 0.0], dtype=np.float32)

    def step(self, action):
        # Декодирование мультидискретного действия в значения gamma и kappa
        if isinstance(action, (np.ndarray, list, tuple)):
            gamma_idx = int(action[0])
            kappa_idx = int(action[1])
        else:
            # Если действие передано как один индекс (например, от DQN), раскодируем его
            action = int(action)
            gamma_idx = action // len(self.kappa_values)
            kappa_idx = action % len(self.kappa_values)
        gamma = self.gamma_values[gamma_idx]
        kappa = self.kappa_values[kappa_idx]
        mid_price = self.prices[self.t]
        # Оценка волатильности (стандартное отклонение относительных изменений цены) на окне последних T секунд
        sigma = 0.0
        if self.t > 1:
            start = max(0, self.t - int(self.T))
            window_prices = self.prices[start : self.t + 1]
            if len(window_prices) > 1:
                returns = np.diff(window_prices) / window_prices[:-1]
                sigma = np.std(returns)
        # Расчёт цены резервирования и оптимального спреда по Avellaneda-Stoikov
        # Цена резервирования учитывает риск (gamma) и текущий инвентарь
        reservation_price = mid_price - self.inventory * gamma * (sigma**2) * self.T
        # Оптимальная половина спреда delta/2 с учётом параметров gamma и kappa
        # Формула: delta = γ * σ^2 * T + (1/γ) * ln(1 + γ/κ)
        delta = gamma * (sigma**2) * self.T + (1.0 / gamma) * math.log(
            1 + gamma / kappa
        )
        bid_price = reservation_price - delta / 2  # цена бид
        ask_price = reservation_price + delta / 2  # цена аск
        done = False
        reward = 0.0
        # Эмуляция исполнения ордеров за следующий шаг
        if self.t < self.N - 1:
            next_price = self.prices[self.t + 1]
            # Если максимальная цена следующего интервала >= нашей ask_price -> ордер на продажу исполнился
            if self.prices_high[self.t + 1] >= ask_price:
                self.inventory -= 1.0
                self.cash += ask_price * (1.0 - self.fee_rate)
            # Если минимальная цена следующего интервала <= нашей bid_price -> ордер на покупку исполнился
            if self.prices_low[self.t + 1] <= bid_price:
                self.inventory += 1.0
                self.cash -= bid_price * (1.0 + self.fee_rate)
            # Рассчитываем новую оценку портфеля и изменение PnL
            current_value = self.cash + self.inventory * next_price
            prev_value = self.cash + self.inventory * mid_price
            pnl_change = current_value - prev_value
            # Небольшой штраф за размер позиции (чтобы ограничивать риск чрезмерного инвентаря)
            reward = pnl_change - 0.001 * abs(self.inventory)
            # Переходим к следующему шагу времени
            self.t += 1
        else:
            # Если достигли конца данных - завершаем эпизод
            done = True
            current_value = self.cash + self.inventory * mid_price
            # Финальный PnL за эпизод является вознаграждением на последнем шаге
            reward = current_value - self.initial_cash
        # Формирование нового состояния
        if not done:
            new_mid = self.prices[self.t]
            price_change = (
                new_mid - mid_price
            ) / mid_price  # относительное изменение цены
            pnl = self.cash + self.inventory * new_mid - self.initial_cash
            obs = np.array([self.inventory, price_change, pnl], dtype=np.float32)
        else:
            obs = None
        return obs, reward, done, {}


### Агенты

In [ ]:
# ==== DQN агент ====

# Q-сеть (критик) для DQN: простая полносвязная сеть
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


# Переход для буфера воспроизведения
Transition = namedtuple(
    "Transition", ["state", "action", "reward", "next_state", "done"]
)


# Буфер воспроизведения для DQN
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def push(self, *args):
        self.buffer.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)


class DQNAgent:
    def __init__(
        self,
        state_dim,
        action_dim,
        lr=1e-3,
        gamma=0.99,
        buffer_capacity=10000,
        batch_size=64,
        target_update=1000,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.q_network = QNetwork(state_dim, action_dim).to(self.device)
        self.target_network = QNetwork(state_dim, action_dim).to(self.device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=lr)
        self.gamma = gamma
        self.buffer = ReplayBuffer(buffer_capacity)
        self.batch_size = batch_size
        self.target_update = target_update
        self.steps_done = 0
        self.action_dim = (
            action_dim  # общее количество возможных действий (комбинаций γ и κ)
        )

    def select_action(self, state, epsilon):
        # ε-жадная стратегия выбора действия
        if random.random() < epsilon:
            # случайное действие (исследование)
            return random.randrange(self.action_dim)
        else:
            # действие по текущей Q-оценке (эксплуатация)
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            with torch.no_grad():
                q_values = self.q_network(state_tensor)
            return q_values.argmax().item()

    def push_transition(self, state, action, reward, next_state, done):
        # Сохраняем переход в буфер
        self.buffer.push(state, action, reward, next_state, done)

    def update(self):
        # Обновление сети DQN из буфера воспроизведения
        if len(self.buffer) < self.batch_size:
            return  # недостаточно данных для обучения
        transitions = self.buffer.sample(self.batch_size)
        batch = Transition(*zip(*transitions))
        # Подготовка тензоров для обучения
        state_batch = torch.FloatTensor(batch.state).to(self.device)
        action_batch = torch.LongTensor(batch.action).unsqueeze(1).to(self.device)
        reward_batch = torch.FloatTensor(batch.reward).unsqueeze(1).to(self.device)
        # Формируем маску для неконечных состояний и тензор следующего состояния
        non_final_mask = torch.tensor(
            [s is not None for s in batch.next_state],
            dtype=torch.bool,
            device=self.device,
        )
        non_final_next_states = torch.FloatTensor(
            [s for s in batch.next_state if s is not None]
        ).to(self.device)
        done_batch = torch.FloatTensor(batch.done).unsqueeze(1).to(self.device)
        # Вычисляем текущие Q(s, a) и целевые Q-значения
        q_values = self.q_network(state_batch).gather(1, action_batch)  # Q(s_t, a_t)
        next_q_values = torch.zeros(self.batch_size, 1).to(self.device)
        if non_final_next_states.size(0) > 0:
            # Максимальное Q для следующего состояния (по целевой сети)
            next_q_values[non_final_mask] = self.target_network(
                non_final_next_states
            ).max(1, keepdim=True)[0]
        expected_q_values = reward_batch + (1 - done_batch) * self.gamma * next_q_values
        # Потеря: MSE между текущим Q и целевым Q
        loss = nn.MSELoss()(q_values, expected_q_values.detach())
        # Обновление Q-сети
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        # Периодическое обновление целевой сети
        self.steps_done += 1
        if self.steps_done % self.target_update == 0:
            self.target_network.load_state_dict(self.q_network.state_dict())


# ==== A2C агент ====


# Актор-критик сеть (общие слои, отдельные выходы actor и critic)
class ActorCritic(nn.Module):
    def __init__(self, input_dim, action_dim):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.actor = nn.Linear(64, action_dim)  # выход для политики (логиты действий)
        self.critic = nn.Linear(64, 1)  # выход для ценности (V(s))

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        logits = self.actor(x)
        value = self.critic(x)
        return logits, value


class A2CAgent:
    def __init__(
        self,
        state_dim,
        action_dim,
        lr=1e-3,
        gamma=0.99,
        value_coef=0.5,
        entropy_coef=0.01,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = ActorCritic(state_dim, action_dim).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef

    def select_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        logits, value = self.model(state_tensor)
        probs = torch.softmax(logits, dim=1)
        # Сэмплируем действие из распределения
        action_tensor = torch.multinomial(probs, num_samples=1)
        log_prob = torch.log(probs.gather(1, action_tensor) + 1e-10)
        # Возвращаем выбранное действие и связанные значения для обучения
        return (
            action_tensor.item(),
            log_prob.squeeze(),
            value.squeeze(),
            logits.squeeze(),
        )

    def update(self, trajectories):
        # trajectories: список кортежей (state, action, log_prob, value, reward, done, logits)
        log_probs = torch.stack([t[2] for t in trajectories]).to(self.device)
        values = torch.stack([t[3] for t in trajectories]).to(self.device)
        rewards = [t[4] for t in trajectories]
        dones = [t[5] for t in trajectories]
        logits_list = torch.stack([t[6] for t in trajectories]).to(self.device)
        # Расчёт накопленных вознаграждений (Returns) методом обратного прохода по траектории
        R = 0
        returns = []
        for reward, done in zip(reversed(rewards), reversed(dones)):
            R = reward + self.gamma * R * (1 - done)
            returns.insert(0, R)
        returns = torch.FloatTensor(returns).to(self.device)
        advantages = returns - values
        # Потери актора и критика
        actor_loss = -(log_probs * advantages.detach()).mean()
        critic_loss = advantages.pow(2).mean()
        # Энтропия политики (для поощрения исследования)
        probs = torch.softmax(logits_list, dim=1)
        entropy = -(probs * torch.log(probs + 1e-10)).sum(dim=1).mean()
        # Суммарный loss
        loss = actor_loss + self.value_coef * critic_loss - self.entropy_coef * entropy
        # Обновление параметров сети
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


# ==== PPO агент ====


class PPOAgent:
    def __init__(
        self,
        state_dim,
        action_dim,
        lr=1e-3,
        gamma=0.99,
        clip_coef=0.2,
        value_coef=0.5,
        entropy_coef=0.01,
        ppo_epochs=4,
    ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = ActorCritic(state_dim, action_dim).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.gamma = gamma
        self.clip_coef = clip_coef  # коэффициент клиппинга PPO (ε)
        self.value_coef = value_coef  # коэффициент для критика
        self.entropy_coef = entropy_coef  # коэффициент для энтропии
        self.ppo_epochs = ppo_epochs  # число эпох обновления на одном наборе траекторий

    def select_action(self, state):
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        logits, value = self.model(state_tensor)
        probs = torch.softmax(logits, dim=1)
        # Семплируем действие из текущей политики
        action_tensor = torch.multinomial(probs, num_samples=1)
        log_prob = torch.log(probs.gather(1, action_tensor) + 1e-10)
        # Возвращаем выбранное действие (скаляр) и связанные величины для хранения траектории
        return action_tensor.item(), log_prob.squeeze(), value.squeeze()

    def update(self, trajectories):
        # trajectories: список кортежей (state, action, log_prob, value, reward, done)
        # Формируем тензоры из собранной траектории
        states = torch.FloatTensor([t[0] for t in trajectories]).to(self.device)
        actions = (
            torch.LongTensor([t[1] for t in trajectories]).unsqueeze(1).to(self.device)
        )
        old_log_probs = torch.stack([t[2] for t in trajectories]).to(self.device)
        old_values = torch.stack([t[3] for t in trajectories]).to(self.device)
        rewards = [t[4] for t in trajectories]
        dones = [t[5] for t in trajectories]
        # Вычисляем массив Returns по траектории (обратным проходом)
        R = 0
        returns = []
        for reward, done in zip(reversed(rewards), reversed(dones)):
            R = reward + self.gamma * R * (1 - done)
            returns.insert(0, R)
        returns = torch.FloatTensor(returns).to(self.device)
        # Вычисляем преимущества (Advantage = Return - baseline)
        # Старые log_prob и value считаем константами на время обновления (без градиентов)
        old_log_probs = old_log_probs.detach()
        old_values = old_values.detach()
        advantages = returns - old_values
        # Необязательная нормализация advantages:
        # advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
        # Многократное обновление политики и ценности на одних и тех же данных траектории (PPO Epochs)
        for _ in range(self.ppo_epochs):
            logits, values = self.model(states)
            values = values.squeeze(1)  # приведение размеров ценности к [batch]
            probs = torch.softmax(logits, dim=1)
            # Новые лог-вероятности выбранных действий по обновленной политике
            new_log_probs = torch.log(probs.gather(1, actions) + 1e-10).squeeze(1)
            # Вычисляем отношение вероятностей π_new/π_old для каждого шага
            ratio = torch.exp(new_log_probs - old_log_probs)
            # Вычисляем функции успеха (surrogate) с клиппингом
            adv_detached = advantages.detach()
            surr1 = ratio * adv_detached
            surr2 = (
                torch.clamp(ratio, 1.0 - self.clip_coef, 1.0 + self.clip_coef)
                * adv_detached
            )
            actor_loss = -torch.min(surr1, surr2).mean()
            # Потеря критика: MSE между прогнозом ценности и подсчитанным возвратом
            critic_loss = (returns - values).pow(2).mean()
            # Энтропия политики (чем выше, тем лучше исследование)
            entropy = -(probs * torch.log(probs + 1e-10)).sum(dim=1).mean()
            # Полный loss с учетом коэффициентов
            loss = (
                actor_loss + self.value_coef * critic_loss - self.entropy_coef * entropy
            )
            # Обновляем параметры модели
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()


### Обучение

In [ ]:
# Задаём дискретные значения параметров gamma и kappa для оптимизации
gamma_values = [0.005, 0.01, 0.05, 0.1, 0.5, 1.0]
kappa_values = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2]

# Создаём окружение с мультидискретными действиями
env = MarketMakingEnv(
    df=df, gamma_values=gamma_values, kappa_values=kappa_values, T=60.0
)
state_dim = env.observation_space.shape[0]
# Определяем размер действия для сетей агентов (количество комбинаций gamma-kappa)
if isinstance(env.action_space, spaces.MultiDiscrete):
    action_dim = int(np.prod(env.action_space.nvec))
else:
    action_dim = env.action_space.n

# Инициализируем агентов DQN, A2C и PPO
dqn_agent = DQNAgent(
    state_dim,
    action_dim,
    lr=1e-3,
    gamma=0.99,
    buffer_capacity=10000,
    batch_size=64,
    target_update=1000,
)
a2c_agent = A2CAgent(
    state_dim, action_dim, lr=1e-3, gamma=0.99, value_coef=0.5, entropy_coef=0.01
)
ppo_agent = PPOAgent(
    state_dim,
    action_dim,
    lr=1e-3,
    gamma=0.99,
    clip_coef=0.2,
    value_coef=0.5,
    entropy_coef=0.01,
    ppo_epochs=4,
)

# Параметры обучения
num_episodes = 20  # эпизодов для DQN
num_episodes_a2c = 20  # эпизодов для A2C
num_episodes_ppo = 20  # эпизодов для PPO
epsilon_start = 1.0
epsilon_final = 0.1
epsilon_decay = 300

# Обучение DQN-агента
print("Обучение DQN-агента...")
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0.0
    done = False
    while not done:
        # Экспоненциальное уменьшение ε в ходе эпизода
        epsilon = epsilon_final + (epsilon_start - epsilon_final) * math.exp(
            -env.t / epsilon_decay
        )
        action = dqn_agent.select_action(state, epsilon)
        next_state, reward, done, _ = env.step(action)
        dqn_agent.push_transition(state, action, reward, next_state, done)
        dqn_agent.update()
        state = next_state if next_state is not None else state
        total_reward += reward
    print(
        f"Эпизод {episode+1}/{num_episodes}: суммарное вознаграждение = {total_reward:.2f}"
    )

# Обучение A2C-агента
print("\nОбучение A2C-агента...")
rollout_length = 10  # длина сегмента траектории перед обновлением
for episode in range(num_episodes_a2c):
    state = env.reset()
    trajectories = []
    total_reward = 0.0
    done = False
    while not done:
        # Сбор траектории длины rollout_length
        for _ in range(rollout_length):
            action, log_prob, value, logits = a2c_agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            # Сохраняем данные шага для последующего обучения
            trajectories.append(
                (state, action, log_prob, value, reward, float(done), logits)
            )
            total_reward += reward
            if done:
                break
            state = next_state
        # Обновляем агент A2C по собранной траектории
        a2c_agent.update(trajectories)
        trajectories = []  # очищаем для сбора следующего сегмента
    print(
        f"Эпизод {episode+1}/{num_episodes_a2c}: суммарное вознаграждение = {total_reward:.2f}"
    )

# Обучение PPO-агента
print("\nОбучение PPO-агента...")
rollout_length_ppo = 20  # длина сегмента траектории для PPO перед обновлением
for episode in range(num_episodes_ppo):
    state = env.reset()
    trajectories = []
    total_reward = 0.0
    done = False
    while not done:
        # Сбор траектории длиной rollout_length_ppo
        for _ in range(rollout_length_ppo):
            action, log_prob, value = ppo_agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            trajectories.append((state, action, log_prob, value, reward, float(done)))
            total_reward += reward
            if done:
                break
            state = next_state
        # Обновляем агент PPO по собранному отрезку траектории (с многократным прогонам PPO)
        ppo_agent.update(trajectories)
        trajectories = []  # очищаем перед сбором следующего сегмента
    print(
        f"Эпизод {episode+1}/{num_episodes_ppo}: суммарное вознаграждение = {total_reward:.2f}"
    )


Обучение DQN-агента...
Эпизод 1/20: суммарное вознаграждение = 1137.85
Эпизод 2/20: суммарное вознаграждение = 1128.52
Эпизод 3/20: суммарное вознаграждение = 457.06
Эпизод 4/20: суммарное вознаграждение = -3.16
Эпизод 5/20: суммарное вознаграждение = 1652.17
Эпизод 6/20: суммарное вознаграждение = -614.21
Эпизод 7/20: суммарное вознаграждение = 656.15
Эпизод 8/20: суммарное вознаграждение = -151.40
Эпизод 9/20: суммарное вознаграждение = 133.54
Эпизод 10/20: суммарное вознаграждение = 967.93
Эпизод 11/20: суммарное вознаграждение = 1510.01
Эпизод 12/20: суммарное вознаграждение = 1347.65
Эпизод 13/20: суммарное вознаграждение = 1610.03
Эпизод 14/20: суммарное вознаграждение = 1336.57
Эпизод 15/20: суммарное вознаграждение = -593.58
Эпизод 16/20: суммарное вознаграждение = 1744.52
Эпизод 17/20: суммарное вознаграждение = 1574.68
Эпизод 18/20: суммарное вознаграждение = 1338.45
Эпизод 19/20: суммарное вознаграждение = 972.15
Эпизод 20/20: суммарное вознаграждение = 1656.64

Обучение A2C

### Тесты

In [8]:
# Функция для прогона тестового эпизода и сбора истории PnL
def run_agent(env, agent, agent_type="dqn"):
    state = env.reset()
    pnl_history = []
    done = False
    while not done:
        if agent_type == "dqn":
            # Для DQN берём действие жадно (epsilon=0 для выбора лучшего действия)
            action = agent.select_action(state, epsilon=0.0)
        elif agent_type == "a2c":
            # Для A2C и PPO можно брать среднее по политике; здесь для простоты тоже сэмплируем
            action, _, _, _ = agent.select_action(state)
        elif agent_type == "ppo":
            action, _, _ = agent.select_action(state)
        else:
            # Базовая стратегия (например, всегда берём первые значения gamma и kappa)
            action = 0
        next_state, _, done, _ = env.step(action)
        # Третий элемент состояния (индекс 2) хранит текущий PnL
        if state is not None:
            pnl_history.append(state[2])
        state = next_state if next_state is not None else state
    return pnl_history


# Тестирование агентов на отложенной выборке
env_test = MarketMakingEnv(
    df=df_test, gamma_values=gamma_values, kappa_values=kappa_values, T=60.0
)
dqn_pnl = run_agent(env_test, dqn_agent, agent_type="dqn")
env_test = MarketMakingEnv(
    df=df_test, gamma_values=gamma_values, kappa_values=kappa_values, T=60.0
)
a2c_pnl = run_agent(env_test, a2c_agent, agent_type="a2c")
env_test = MarketMakingEnv(
    df=df_test, gamma_values=gamma_values, kappa_values=kappa_values, T=60.0
)
ppo_pnl = run_agent(env_test, ppo_agent, agent_type="ppo")


In [9]:
def compute_max_drawdown(pnl_history):
    pnl_array = np.array(pnl_history)
    running_max = np.maximum.accumulate(pnl_array)
    drawdowns = pnl_array - running_max
    max_drawdown = drawdowns.min()
    return max_drawdown


def compute_sharpe_ratio(pnl_history):
    pnl_array = np.array(pnl_history)
    returns = np.diff(pnl_array)
    if returns.std() == 0:
        return 0.0
    sharpe = returns.mean() / returns.std() * np.sqrt(len(returns))
    return sharpe


dqn_max_dd = compute_max_drawdown(dqn_pnl)
dqn_sharpe = compute_sharpe_ratio(dqn_pnl)
a2c_max_dd = compute_max_drawdown(a2c_pnl)
a2c_sharpe = compute_sharpe_ratio(a2c_pnl)
ppo_max_dd = compute_max_drawdown(ppo_pnl)
ppo_sharpe = compute_sharpe_ratio(ppo_pnl)

print("\nРезультаты тестирования:")
print(
    f"DQN итоговый PnL: {dqn_pnl[-1]:.2f}, Max Drawdown: {dqn_max_dd:.2f}, Sharpe Ratio: {dqn_sharpe:.2f}"
)
print(
    f"A2C итоговый PnL: {a2c_pnl[-1]:.2f}, Max Drawdown: {a2c_max_dd:.2f}, Sharpe Ratio: {a2c_sharpe:.2f}"
)
print(
    f"PPO итоговый PnL: {ppo_pnl[-1]:.2f}, Max Drawdown: {ppo_max_dd:.2f}, Sharpe Ratio: {ppo_sharpe:.2f}"
)


Результаты тестирования:
DQN итоговый PnL: 1.63, Max Drawdown: -16.43, Sharpe Ratio: 0.07
A2C итоговый PnL: 25.61, Max Drawdown: -21.30, Sharpe Ratio: 0.87
PPO итоговый PnL: -41.29, Max Drawdown: -81.20, Sharpe Ratio: -0.56
